[Reference](https://medium.com/@mansha99/concurrency-in-python-threads-and-locks-8daed86e92e6)

In [1]:
import threading
t = threading.current_thread()
print(t.name)
print(t.native_id)

MainThread
346


In [2]:
import threading
import time
loop_count=10
class PrintDotThread(threading.Thread):
    def run(self) -> None:
        for x in range(loop_count):
            print("." ,end="")
            time.sleep(10/1000)#10 millisecond
        print("")
        return super().run()

#main program
print("BEGIN")
printDotThread=PrintDotThread()
printDotThread.start()
print("")
print("END")

BEGIN
.
END


In [3]:
import threading
import time
loop_count=10
class PrintDotThread(threading.Thread):
    def run(self) -> None:
        for x in range(loop_count):
            print("." ,end="")
            time.sleep(10/1000)#10 millisecond
        print("")
        return super().run()

#main program
print("BEGIN")
printDotThread=PrintDotThread()
printDotThread.start()
printDotThread.join();# <------------
print("")
print("END")

BEGIN
..........

END


In [4]:
import threading
import time

#shared object
class BankAccount:
    def __init__(self) -> None:
        self.balance = 0

    def deposit(self,amount,customer_name):
        print("%s Depositing amount : %d , Current Balance : %d" %(customer_name, amount,self.balance))
        time.sleep(10/1000)
        self.balance = self.balance + amount
        print("%s Checking Balance : its  %d" %(customer_name, self.balance))

#concurrent access: Thread
class CustomerThread(threading.Thread):
    def __init__(self, customer_name:str,account:BankAccount):
        super(CustomerThread, self).__init__()
        self.customer_name=customer_name
        self.account=account
    def run(self) -> None:
        self.account.deposit(1000,self.customer_name)
        return super().run()

#main program
account=BankAccount()
customer1=CustomerThread("Smith",account)
customer2=CustomerThread("Steves",account)
customer1.start()
customer2.start()

Smith Depositing amount : 1000 , Current Balance : 0
Steves Depositing amount : 1000 , Current Balance : 0
Smith Checking Balance : its  1000
Steves Checking Balance : its  2000


In [5]:
import threading
import time

class BankAccount:
    def __init__(self) -> None:
        self.balance = 0
        self.lock = threading.Lock() # create a lock

    def deposit(self,amount,customer_name):
        self.lock.acquire() # begin critical section
        print("%s Depositing amount : %d , Current Balance : %d" %(customer_name, amount,self.balance))
        time.sleep(10/1000)
        self.balance = self.balance + amount
        print("%s Checking Balance : its  %d" %(customer_name, self.balance))
        self.lock.release() # end critical section

class CustomerThread(threading.Thread):
    def __init__(self, customer_name:str,account:BankAccount):
        super(CustomerThread, self).__init__()
        self.customer_name=customer_name
        self.account=account
    def run(self) -> None:
        self.account.deposit(1000,self.customer_name)
        return super().run()

account=BankAccount()
customer1=CustomerThread("Smith",account)
customer2=CustomerThread("Steves",account)
customer1.start()
customer2.start()

Smith Depositing amount : 1000 , Current Balance : 0


In [6]:
import threading
import time

class BankAccount:
    def __init__(self) -> None:
        self.balance = 0
        self.lock = threading.RLock() # <---- This is recursive reentrant lock

    def deposit(self,amount,customer_name):
        self.lock.acquire() # lock
        self.lock.acquire() # again and again
        print("%s Depositing amount : %d , Current Balance : %d" %(customer_name, amount,self.balance))
        time.sleep(10/1000)
        self.balance = self.balance + amount
        print("%s Checking Balance : its  %d" %(customer_name, self.balance))
        self.lock.release() #release
        self.lock.release() #release

class CustomerThread(threading.Thread):
    def __init__(self, customer_name:str,account:BankAccount):
        super(CustomerThread, self).__init__()
        self.customer_name=customer_name
        self.account=account
    def run(self) -> None:
        self.account.deposit(1000,self.customer_name)
        return super().run()

account=BankAccount()
customer1=CustomerThread("Smith",account)
customer2=CustomerThread("Steves",account)
customer1.start()
customer2.start()

Smith Depositing amount : 1000 , Current Balance : 0
